In [1]:
# Import relevant frameworks and libraries
import datetime
import hmac
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
import base64
import os

In [2]:
# Case 1: Digital Signature (DS)
# SecurityDemo Class when using Digital Signature
class SecurityDemo:
    def __init__(self):
        # Generate RSA key pair for Digital Signature
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048
        )
        self.public_key = self.private_key.public_key()

    # Implementing Digital Signature verification against replay attack
    def digital_signature(self, message):
        """Demonstrate digital signature implementation"""
        print("\n=== Digital Signature Implementation ===")
        print(f"Original message: {message}")
        
        # Alice signs the message
        signature = self.private_key.sign(
            message.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        signature_b64 = base64.b64encode(signature).decode()
        print(f"Digital Signature: {signature_b64[:50]}...")  # Display only the first 50 characters of the digital signature
        
        # Oscar captures and replays the message 100 times
        print("\nOscar captures and replays the message 100 times:")
        for i in range(100):
            try:
                # Bob verifies Digital Signature
                self.public_key.verify(
                    signature,
                    message.encode(),
                    padding.PSS(
                        mgf=padding.MGF1(hashes.SHA256()),
                        salt_length=padding.PSS.MAX_LENGTH
                    ),
                    hashes.SHA256()
                )
                print(f"Replay {i+1}: Digital Signature verified successfully!")
            except Exception as e:
                print(f"Replay {i+1}: Verification failed!") 
    
# Execute the demonstration
if __name__ == "__main__":
    demo = SecurityDemo()
    message = "Transfer $1000 to Oscar"
    
    # Demonstrate the Digital Signature implementation
    demo.digital_signature(message)


=== Digital Signature Implementation ===
Original message: Transfer $1000 to Oscar
Digital Signature: bkbi7lAlOOo+o3LRHxzQjPWAX7otXDrIEGYTDFXUQtyZlIknTD...

Oscar captures and replays the message 100 times:
Replay 1: Digital Signature verified successfully!
Replay 2: Digital Signature verified successfully!
Replay 3: Digital Signature verified successfully!
Replay 4: Digital Signature verified successfully!
Replay 5: Digital Signature verified successfully!
Replay 6: Digital Signature verified successfully!
Replay 7: Digital Signature verified successfully!
Replay 8: Digital Signature verified successfully!
Replay 9: Digital Signature verified successfully!
Replay 10: Digital Signature verified successfully!
Replay 11: Digital Signature verified successfully!
Replay 12: Digital Signature verified successfully!
Replay 13: Digital Signature verified successfully!
Replay 14: Digital Signature verified successfully!
Replay 15: Digital Signature verified successfully!
Replay 16: Digital Si

In [3]:
# Case 2: Message Authentication Code (MAC)
# SecurityDemo Class when using Message Authentication Code
class SecurityDemo:
    def __init__(self):
        # Generate shared secret key for Message Authentication Code
        self.mac_key = b"shared_secret_key_between_alice_and_bob"
        
    # Implementing Message Authentication Code verification against replay attack
    def message_authentication_code(self, message):
        """Demonstrate Message Authentication Code implementation"""
        print("\n=== Message Authentication Code Implementation ===")
        print(f"Original message: {message}")
        
        # Alice generates Message Authentication Code
        mac = hmac.new(self.mac_key, message.encode(), hashlib.sha256).digest()
        mac_b64 = base64.b64encode(mac).decode()
        print(f"Message Authentication Code: {mac_b64}")
        
        # Oscar captures and replays the message 100 times
        print("\nOscar captures and replays the message 100 times:")
        for i in range(100):
            # Bob verifies Message Authentication Code
            received_mac = hmac.new(self.mac_key, message.encode(), hashlib.sha256).digest()
            if hmac.compare_digest(mac, received_mac):
                print(f"Replay {i+1}: Message Authentication Code verified successfully!")
            else:
                print(f"Replay {i+1}: Message Authentication Code verification failed!")

# Execute the demonstration
if __name__ == "__main__":
    demo = SecurityDemo()
    message = "Transfer $1000 to Oscar"
    
    # Demonstrate the Message Authentication Code implementation
    demo.message_authentication_code(message)


=== Message Authentication Code Implementation ===
Original message: Transfer $1000 to Oscar
Message Authentication Code: WpZP5C1TuWru98kiCsIrOlYWwItFLACaW7EAkKZkBOU=

Oscar captures and replays the message 100 times:
Replay 1: Message Authentication Code verified successfully!
Replay 2: Message Authentication Code verified successfully!
Replay 3: Message Authentication Code verified successfully!
Replay 4: Message Authentication Code verified successfully!
Replay 5: Message Authentication Code verified successfully!
Replay 6: Message Authentication Code verified successfully!
Replay 7: Message Authentication Code verified successfully!
Replay 8: Message Authentication Code verified successfully!
Replay 9: Message Authentication Code verified successfully!
Replay 10: Message Authentication Code verified successfully!
Replay 11: Message Authentication Code verified successfully!
Replay 12: Message Authentication Code verified successfully!
Replay 13: Message Authentication Code verifie

In [4]:
# Case 1: Digital Signature (DS)
# SecurityDemo Class for replay prevention by improving Digital Signature
class SecurityDemo:
    def __init__(self):
        # Generate RSA key pair for Digital Signature
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048
        )
        self.public_key = self.private_key.public_key()

    # Store for used timestamps to prevent replay
        self.used_timestamps = set()

    # Improved Digital Signature implementation with timestamp
    def digital_signature_improved(self, message):
        """Demonstrate improved Digital Signature implementation using timestamps"""
        print("\n=== Improved Digital Signature Implementation (with Timestamp) ===")
        print(f"Original message: {message}")
        
        # Alice signs message with timestamp
        timestamp = datetime.datetime.now().isoformat()
        message_with_timestamp = f"{message}|{timestamp}"
        print(f"Message with timestamp: {message_with_timestamp}")
        
        signature = self.private_key.sign(
            message_with_timestamp.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        
        # Oscar captures and replays the message 100 times
        print("\nOscar captures and replays the message 100 times:")
        for i in range(100):
            try:
                # Bob checks if timestamp was used before
                if timestamp in self.used_timestamps:
                    print(f"Replay {i+1}: Rejected - Timestamp already used!")
                    continue
                    
                # Bob verifies Digital Signature
                self.public_key.verify(
                    signature,
                    message_with_timestamp.encode(),
                    padding.PSS(
                        mgf=padding.MGF1(hashes.SHA256()),
                        salt_length=padding.PSS.MAX_LENGTH
                    ),
                    hashes.SHA256()
                )
                
                # Store timestamp after successful verification
                self.used_timestamps.add(timestamp)
                print(f"Replay {i+1}: Digital Signature verified successfully! First use accepted, timestamp stored!!")
            except Exception as e:
                print(f"Replay {i+1}: Verification failed!")

# Execute the demonstration
if __name__ == "__main__":
    demo = SecurityDemo()
    message = "Transfer $1000 to Oscar"
    
    # Demonstrate the improved Digital Signature implementation
    demo.digital_signature_improved(message)


=== Improved Digital Signature Implementation (with Timestamp) ===
Original message: Transfer $1000 to Oscar
Message with timestamp: Transfer $1000 to Oscar|2024-11-07T17:21:17.805688

Oscar captures and replays the message 100 times:
Replay 1: Digital Signature verified successfully! First use accepted, timestamp stored!!
Replay 2: Rejected - Timestamp already used!
Replay 3: Rejected - Timestamp already used!
Replay 4: Rejected - Timestamp already used!
Replay 5: Rejected - Timestamp already used!
Replay 6: Rejected - Timestamp already used!
Replay 7: Rejected - Timestamp already used!
Replay 8: Rejected - Timestamp already used!
Replay 9: Rejected - Timestamp already used!
Replay 10: Rejected - Timestamp already used!
Replay 11: Rejected - Timestamp already used!
Replay 12: Rejected - Timestamp already used!
Replay 13: Rejected - Timestamp already used!
Replay 14: Rejected - Timestamp already used!
Replay 15: Rejected - Timestamp already used!
Replay 16: Rejected - Timestamp alread

In [5]:
# Case 2: Message Authentication Code (MAC)
# SecurityDemo Class for replay prevention by improving Message Authentication Code
class SecurityDemo:
    def __init__(self):
        # Generate shared secret key for Message Authentication Code
        self.mac_key = b"shared_secret_key_between_alice_and_bob"

    # Store for used nonces to prevent replay
        self.used_nonces = set()

    # Improved Message Authentication Code implementation with nonce
    def message_authentication_code_improved(self, message):
        """Demonstrate improved Message Authentication Code implementation using nonces"""
        print("\n=== Improved Message Authentication Code Implementation (with Nonce) ===")
        print(f"Original message: {message}")
        
        # Alice generates nonce and Message Authentication Code
        nonce = base64.b64encode(os.urandom(16)).decode()
        message_with_nonce = f"{message}|{nonce}"
        print(f"Message with nonce: {message_with_nonce}")
        
        mac = hmac.new(self.mac_key, message_with_nonce.encode(), hashlib.sha256).digest()
        
        # Oscar captures and replays the message 100 times
        print("\nOscar captures and replays the message 100 times:")
        for i in range(100):
            # Bob checks if nonce was used before
            if nonce in self.used_nonces:
                print(f"Replay {i+1}: Rejected - Nonce already used!")
                continue
                
            # Bob verifies Message Authentication Code
            received_mac = hmac.new(self.mac_key, message_with_nonce.encode(), hashlib.sha256).digest()
            if hmac.compare_digest(mac, received_mac):
                # Store nonce after successful verification
                self.used_nonces.add(nonce)
                print(f"Replay {i+1}: Message Authentication Code verified successfully! First use accepted, nonce stored!!")
            else:
                print(f"Replay {i+1}: Message Authentication Code verification failed!")

# Execute the demonstration
if __name__ == "__main__":
    demo = SecurityDemo()
    message = "Transfer $1000 to Oscar"
    
    # Demonstrate the improved Message Authentication Code implementation
    demo.message_authentication_code_improved(message)


=== Improved Message Authentication Code Implementation (with Nonce) ===
Original message: Transfer $1000 to Oscar
Message with nonce: Transfer $1000 to Oscar|k0HuV4DkExD8trUhduwldg==

Oscar captures and replays the message 100 times:
Replay 1: Message Authentication Code verified successfully! First use accepted, nonce stored!!
Replay 2: Rejected - Nonce already used!
Replay 3: Rejected - Nonce already used!
Replay 4: Rejected - Nonce already used!
Replay 5: Rejected - Nonce already used!
Replay 6: Rejected - Nonce already used!
Replay 7: Rejected - Nonce already used!
Replay 8: Rejected - Nonce already used!
Replay 9: Rejected - Nonce already used!
Replay 10: Rejected - Nonce already used!
Replay 11: Rejected - Nonce already used!
Replay 12: Rejected - Nonce already used!
Replay 13: Rejected - Nonce already used!
Replay 14: Rejected - Nonce already used!
Replay 15: Rejected - Nonce already used!
Replay 16: Rejected - Nonce already used!
Replay 17: Rejected - Nonce already used!
Repl